# Datojen haku ja esikäsittely

In [ ]:
from set_path import set_path
mainpath, path = set_path('areadata')

In [ ]:
from read_and_prepare_data import read_and_prepare_data
stat, post, kunta_stat, vaalidata = read_and_prepare_data(path)

In [ ]:
from selected_cols import selected_cols
numeric_features, categorical_features = selected_cols(largeset=False, parties=True)

In [ ]:
from sklearn.feature_selection import  mutual_info_classif
import pandas as pd
import numpy as np
from draw_and_create_clusters import create_kmeans_clusters 

from prepare_and_scale_data import prepare_and_scale_data
from create_prediction import select_kbest
from select_columns_and_clean_data import select_columns_and_clean_data
from draw_all import draw_features
from create_dbscan_clusters import create_dbscan_clusters


%matplotlib inline
%config InlineBackend.figure_format = 'retina'
pd.options.display.max_colwidth = 100


#valitaan käsittelyyn tuleva alue 91 = Helsinki
selected = stat[(stat['muncipality_code']==91) | (stat['muncipality_code']==92) | (stat['muncipality_code']==49) | (stat['muncipality_code']==235)].copy()
selected_vaalidata = vaalidata[(vaalidata['Kuntanumero']==91) | (vaalidata['Kuntanumero']==92) | (vaalidata['Kuntanumero']==49) | (vaalidata['Kuntanumero']==235) ].copy()
post_included = post[(post['muncipality_code']==91) | (post['muncipality_code']==92) | (post['muncipality_code']==49) | (post['muncipality_code']==235)]['postcode'].copy()

#valitaan käsittelyyn tuleva alue FI1B1 = Uusimaa
selected = stat[stat['area_code']=='FI1B1'].copy()
selected_vaalidata = vaalidata[vaalidata['area_code']=='FI1B1'].copy()
post_included = post[post['area_code']=='FI1B1'].copy()


selected = stat.copy()
selected_vaalidata = vaalidata.copy()
post_included = post.copy()



In [ ]:
from gaussian_model import create_and_draw_best_gaussian_mix

data, X, test, gmm_bic, gmm_aic = create_and_draw_best_gaussian_mix(selected, selected, numeric_features=numeric_features, categorical_features=categorical_features, scaled=True, covariance_type = 'full')



In [ ]:
from gaussian_model import create_gaussian_model
data = create_gaussian_model(data, X, gmm_bic)

In [ ]:
data['cluster'].value_counts()

In [ ]:
import geopandas
from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list('my cmap', ['gold', 'green', 'blue', 'pink', 'gold', 'darkgreen', 'lightgreen'])
draw = geopandas.GeoDataFrame(pd.DataFrame(data[['cluster', 'geometry']], dtype=object),geometry='geometry')
draw.plot(column='cluster', cmap=cmap, legend=True, figsize=(20,20))

In [ ]:
import xgboost as xgb
from sklearn.feature_selection import  mutual_info_classif
from create_prediction import create_prediction

target = 'cluster'

ennustettu = "uusi" + target

#Check
#k_selected need to have correct values, if they are too large, evrything needs to be started again
k_selected = 'all'

test_size = 0.2

#Scalers attributes
kbest_score_func = mutual_info_classif
#set values to model configuration attributes

num_class = len(data['cluster'].unique()) #how many classes we are working with

metric = 'mlogloss'
Skfold=True
Verbose = False
testing=True

params = {
    # Parameters that we are going to tune.
    'objective':'multi:softprob',
    'num_class' : num_class,
    'booster' : 'gbtree',
    'eval_metric' : metric
} 

data, test, features_df, importance_df, model, params, y_pred, X_train, y_train, X_test, y_test = create_prediction(data, data, target, kbest_score_func, metric, params, numeric_features, categorical_features, scaled=False, testing=testing)
data_no_use, X_scale, X = prepare_and_scale_data(data, pd.DataFrame(), numeric_features, categorical_features)


In [ ]:
mymodel, explainer, shap_values, shap_interaction_values, shap_expected_value = shap_Xboost(data, target, params, X)

In [ ]:
shap.summary_plot(shap_values, X,  max_display=25)

In [ ]:
import shap
test=shap.summary_plot(shap_values, X,  max_display=25)

In [ ]:
pnro = '02160'
id_col = 'Postinumero'
df, X_rand, clst = shap_proba_individual(explainer, mymodel, data, target, X, id_col, pnro)
print_individual(df,pnro, clst)

In [ ]:
print_reason(X_rand, explainer, shap_values, mymodel, clst)

In [ ]:
print_individual_waterfall(data, target, X, shap_expected_value, shap_values, id_col, pnro)

In [ ]:
Party = 0

vals = shap_vals(X, "Party " + class_names[Party], shap_values[0], n=20)

In [ ]:
new = ['Postinumero','muncipality_name','Asukkaat yhteensä, 2018 (HE)', target]
show_df, col_list = print_examples(data, X, new, target, cluster, shap_values, col_num=9, n=6)
show_df

In [ ]:
shap.summary_plot(shap_interaction_values[cluster], X)